In [ ]:
import sys
import cmath
import math
import os
import h5py
import matplotlib.pyplot as plt   # plots
from matplotlib.ticker import MaxNLocator
import numpy as np
import time
import warnings
import glob
from liblibra_core import *
import util.libutil as comn
from libra_py import units
import models
import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot
import libra_py.data_savers as data_savers

from recipes import fssh, fssh2, fssh3, gfsh

import libra_py.models.GLVC as GLVC


warnings.filterwarnings('ignore')

In [ ]:
xls = [50.0]
ntraj = 1000
istate = 1
elec_int = 2
rep = 1
dt = 5.0
dpi = 300
print(f'{dt} {elec_int}')
eps_vals = [0.01, 0.05, 0.1, 0.4, 1.0,10.0, 50.0, 100.0, 500.0, 1000.0]
A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 
          1000000.0, 10000000.0, 100000000.0, 1000000000.0]

In [ ]:
from libra_py import units
import numpy as np
s = units.inv_cm2Ha
freqs = np.array([129.743484,263.643268,406.405874,564.000564,744.784527,
                  961.545250,1235.657107,1606.622430,2157.558683,3100.000000])*s
freqs = freqs.tolist()
g_vals = [0.000563,0.001143,0.001762,0.002446,0.003230,0.004170,0.005358,0.006966,0.009356,0.013444]
eps = 0.02
v0 = 0.001
m = 1836.0 # 1 amu
temperature = 300 # K
model_params = {"model": 0, "model0": 0,"nstates": 2, "num_osc": 10, "coupling_scaling": [1.0, -1.0],
                "omega": [freqs]*2, "coupl": [g_vals]*2, "mass": [m]*10, "Ham": [[eps,-v0],[-v0,-eps]],
                "beta": 1/(units.kB*temperature)}
_nosc = 10
beta = model_params["beta"]
#=========== Boltzmann
q_width_boltzmann = [ 1.0/( np.sqrt(model_params["mass"][i] * beta) * model_params["omega"][0][i]) for i in range(_nosc)],
p_width_boltzmann = [ 1.0/( np.sqrt(beta/model_params["mass"][i])) for i in range(_nosc)],
#=========== Wigner low-temperature limit (Saikat's)
q_width_wigner_saikat = [ 1.0/np.sqrt(model_params["mass"][i] * model_params["omega"][0][i]) for i in range(_nosc)],
p_width_wigner_saikat = [ np.sqrt( model_params["mass"][i] * model_params["omega"][0][i]  ) for i in range(_nosc)],
#=========== Wigner finite temperature
q_width_wigner_300 = [ 1.0/(np.sqrt(model_params["mass"][i] * model_params["omega"][0][i] * np.tanh(beta*model_params["omega"][0][i]/2) )) for i in range(_nosc)],
p_width_wigner_300 = [ np.sqrt( model_params["mass"][i] * model_params["omega"][0][i] / np.tanh(beta*model_params["omega"][0][i]/2) ) for i in range(_nosc)],

In [ ]:
print(list(p_width_boltzmann))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# x = np.arange(len(q_width_boltzmann))  # x positions

plt.rcParams.update({'font.size': 35, 'axes.linewidth': 3, 'lines.linewidth': 3.0})
plt.figure(figsize=(3.21*4,2.41*3))
x = np.arange(10)
x_labels = [
    "$\omega_1$", "$\omega_2$", "$\omega_3$", "$\omega_4$", "$\omega_5$",
    "$\omega_6$", "$\omega_7$", "$\omega_8$", "$\omega_9$", "$\omega_{10}$"
]

plt.bar(x, list(q_width_boltzmann)[0], width=0.2, label='Boltzmann 300K', color='blue')#, color='#e6dff1')
plt.bar(x+0.3, list(q_width_wigner_saikat)[0], width=0.2, label='Wigner 0K', color='red', alpha=0.8)
plt.bar(x+0.6, list(q_width_wigner_300)[0], width=0.2, label='Wigner 300K', color='black', alpha=0.8)
plt.ylabel("$\sigma_q$, a.u.")
plt.xticks(x+0.3, x_labels)
plt.legend()
plt.xlabel('Frequency')
# plt.show()
plt.tight_layout()
plt.savefig('sigma_q_distribution.jpg', dpi=600)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# x = np.arange(len(q_width_boltzmann))  # x positions

plt.rcParams.update({'font.size': 35, 'axes.linewidth': 3, 'lines.linewidth': 3.0})
plt.figure(figsize=(3.21*4,2.41*3))
x = np.arange(10)
x_labels = [
    "$\omega_1$", "$\omega_2$", "$\omega_3$", "$\omega_4$", "$\omega_5$",
    "$\omega_6$", "$\omega_7$", "$\omega_8$", "$\omega_9$", "$\omega_{10}$"
]

plt.bar(x, list(p_width_boltzmann)[0], width=0.2, label='Boltzmann 300K', color='blue')#, color='#e6dff1')
plt.bar(x+0.3, list(p_width_wigner_saikat)[0], width=0.2, label='Wigner 0K', color='red', alpha=0.8)
plt.bar(x+0.6, list(p_width_wigner_300)[0], width=0.2, label='Wigner 300K', color='black', alpha=0.8)
plt.ylabel("$\sigma_p$, a.u.")
plt.xticks(x+0.3, x_labels)
plt.xlabel('Frequency')
plt.legend()
plt.tight_layout()
# plt.show()
plt.savefig('sigma_p_distribution.jpg', dpi=600)

In [ ]:
def compute_zpe(omega):
    zpe = (1/2) * np.sum(omega)
    return zpe

zpe = compute_zpe(model_params["omega"])
print('ZPE is:',zpe)

In [ ]:
import h5py

In [ ]:
Etot_ave = []
for sampling in ['boltzmann', 'wigner0', 'wigner_finite']:
    etot_ave = []
    for ntraj in [10, 100, 500, 1000, 2000, 5000, 10000]:
        file_name = f'convergence_test/fssh__ntraj_{ntraj}_iter_0_dt_1.0_istate_1_rep_1_elec_int_2_{sampling}/mem_data.hdf'
        F = h5py.File(file_name)
        value = np.array(F['Etot_ave/data'])[0]
        F.close()
        etot_ave.append(value)
    Etot_ave.append(etot_ave)
print(Etot_ave)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 35, 'axes.linewidth': 3, 'lines.linewidth': 3.0})
plt.figure(figsize=(3.21*5,2.41*4))
x = np.arange(7)
x_labels = [f'{ntraj}' for ntraj in [10, 100, 500, 1000, 2000, 5000, 10000]]

plt.bar(x, Etot_ave[0], width=0.2, label='Boltzmann 300K', color='blue')#, color='#e6dff1')
plt.bar(x+0.3, Etot_ave[1], width=0.2, label='Wigner 0K', color='red', alpha=0.8)
plt.bar(x+0.6, Etot_ave[2], width=0.2, label='Wigner 300K', color='black', alpha=0.8)
plt.xticks(x+0.3, x_labels, rotation=90)
plt.xlabel('Number of SH trajectories')
plt.ylabel('Average total energy, a.u.')
plt.yscale('log')
# plt.xscale('log')
plt.title('Convergence analysis')
plt.legend(loc='upper left')
plt.tight_layout()
plt.savefig('convergence_analysis.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'fssh'
filename_1fs = f'all_methods_dec_time_1.0/{method}__ntraj_100_iter_0_dt_1.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/{method}__ntraj_100_iter_0_dt_5.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('FSSH')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_1.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'fssh'
filename_1fs = f'all_methods_dec_time_1.0/fssh_SDM_EDC_ntraj_100_iter_0_dt_1.0_eps_param_10.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/fssh_SDM_EDC_ntraj_100_iter_0_dt_5.0_eps_param_10.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('FSSH-SDM-EDC')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_2.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'fssh'
filename_1fs = f'all_methods_dec_time_1.0/fssh_SHXF_ntraj_100_iter_0_dt_1.0_wpwidth_scale_0.25_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/fssh_SHXF_ntraj_100_iter_0_dt_5.0_wpwidth_scale_0.25_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('FSSH-XF')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_3.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'fssh'
filename_1fs = f'all_methods_dec_time_1.0/gfsh_SHXF_ntraj_100_iter_0_dt_1.0_wpwidth_scale_0.25_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/gfsh_SHXF_ntraj_100_iter_0_dt_5.0_wpwidth_scale_0.25_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('GFSH-XF')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_4.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'fssh'
filename_1fs = f'all_methods_dec_time_1.0/gfsh_SDM_EDC_ntraj_100_iter_0_dt_1.0_eps_param_5.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/gfsh_SDM_EDC_ntraj_100_iter_0_dt_5.0_eps_param_5.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('GFSH-SDM-EDC')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_5.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'gfsh'
filename_1fs = f'all_methods_dec_time_1.0/{method}__ntraj_100_iter_0_dt_1.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/{method}__ntraj_100_iter_0_dt_5.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('GFSH')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_6.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'fssh2'
filename_1fs = f'all_methods_dec_time_1.0/{method}__ntraj_100_iter_0_dt_1.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/{method}__ntraj_100_iter_0_dt_5.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('FSSH2')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_7.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'fssh2'
filename_1fs = f'all_methods_dec_time_1.0/fssh2_SDM_EDC_ntraj_100_iter_0_dt_1.0_eps_param_5.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/fssh2_SDM_EDC_ntraj_100_iter_0_dt_5.0_eps_param_5.0_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('FSSH2-SDM-EDC')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_8.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
plt.figure(figsize=(3.21*4.5, 2.41*4))

method = 'fssh2'
filename_1fs = f'all_methods_dec_time_1.0/fssh2_SHXF_ntraj_100_iter_0_dt_1.0_wpwidth_scale_0.25_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x = np.array(F[f'sh_pop_adi/data'])
time_vec = F['time/data'][:]*units.au2fs/1000
F.close()

filename_5fs = f'all_methods_dec_time_5.0/fssh2_SHXF_ntraj_100_iter_0_dt_5.0_wpwidth_scale_0.25_istate_1_rep_1_elec_int_2/mem_data.hdf'
F = h5py.File(filename_1fs)
x5 = np.array(F[f'sh_pop_adi/data'])
time_vec5 = F['time/data'][:]*units.au2fs/1000
F.close()
plt.title('FSSH2-XF')
plt.plot(time_vec, x[:,1], label=f'1.0 a.u.')
plt.plot(time_vec5, x5[:,1], ls='dotted', label=f'5.0 a.u.')
plt.legend()
plt.ylabel('S$_1$ SH Population')
plt.xlabel('Time, ps')
plt.xlim(0,50)
plt.tight_layout()
plt.savefig('time_step_comparison_9.jpg', dpi=600)

In [ ]:
np.load('wigner_0K_errors_sorted_folders_no_ssy.npy')

In [ ]:
np.load('wigner_0K_errors_sorted_folders_with_ssy.npy')

In [ ]:
np.load('boltzmann_errors_sorted_folders_no_ssy.npy')

In [ ]:
np.load('boltzmann_errors_sorted_folders_with_ssy.npy')

In [ ]:
# file = 'all_methods_dec_2/fssh2_DISH_REV23_EDC_ntraj_1000_iter_0_dt_5.0_eps_param_0.01_SSY_istate_1_rep_1_elec_int_2/mem_data.hdf'
for c, file in enumerate(glob.glob('all_methods_dec_2/*/*.hdf')):
    print(c, file)
    F = h5py.File(file)
    pop = np.array(F['sh_pop_adi/data'])
    if c==0:
        time = np.array(F['time/data'])
    F.close()
    np.savetxt(f"outputs_text/{file.replace('all_methods_dec_2/','').replace('/mem_data.hdf','')}"+"_boltzmann.txt", pop, fmt="%.5f")
    if c==0:
        np.savetxt('outputs_text/time.txt', time, fmt="%.5f")

In [ ]:
# file = 'all_methods_dec_2/fssh2_DISH_REV23_EDC_ntraj_1000_iter_0_dt_5.0_eps_param_0.01_SSY_istate_1_rep_1_elec_int_2/mem_data.hdf'
for c, file in enumerate(glob.glob('all_methods_dec_wigner_saikat/*/*.hdf')):
    print(c, file)
    F = h5py.File(file)
    pop = np.array(F['sh_pop_adi/data'])
    if c==0:
        time = np.array(F['time/data'])
    F.close()
    np.savetxt(f"outputs_text/{file.replace('all_methods_dec_wigner_saikat/','').replace('/mem_data.hdf','')}"+"_wigner_0K.txt", pop, fmt="%.5f")
    if c==0:
        np.savetxt('outputs_text/time.txt', time, fmt="%.5f")